In [11]:
# 0. Imports
from __future__ import annotations
import os, random, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

import mediapipe as mp

# Replace this failing line:
# from mediapipe.solutions.pose import PoseLandmark

# With:
PoseLandmark = mp.solutions.pose.PoseLandmark

# Then:
JOINT_NAMES = [lm.name for lm in PoseLandmark]
N_JOINTS    = len(JOINT_NAMES)  # should be 33

print(f"JOINT_NAMES: {JOINT_NAMES}")
print(f"N_JOINTS: {N_JOINTS}")

JOINT_NAMES: ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
N_JOINTS: 33


In [ ]:
# 1. Paths & device
SCRIPT_DIR    = Path().resolve()
DATA_ROOT     = SCRIPT_DIR/"Data-REHAB24-6"
WIN_CSV       = DATA_ROOT/"Segmentation_windows.csv"
KEYPT_ROOT    = DATA_ROOT/"mp_keypoints"

DEVICE = (
    torch.device("mps") if torch.backends.mps.is_available() else
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("cpu")
)
print("► Using device:", DEVICE)

► Using device: mps


In [ ]:
# 2. Joint names & count
PoseLandmark = mp.solutions.pose.PoseLandmark
JOINT_NAMES  = [lm.name for lm in PoseLandmark]
ERR_JOINTS   = [
  "LEFT_ELBOW","RIGHT_ELBOW",
  "LEFT_SHOULDER","RIGHT_SHOULDER",
  "LEFT_HIP","RIGHT_HIP",
  "LEFT_KNEE","RIGHT_KNEE",
  "SPINE","HEAD",
]
N_ERR = len(ERR_JOINTS)   # 10
ERR_COLS = [f"err_{i}" for i in range(N_ERR)]


✔ Saved MediaPipe joint names to /Users/jithinkrishnan/Documents/Study/IS06 /MVP/RehabApp/model-training-scripts/Data-REHAB24-6/mp_keypoints/mediapipe_joint_names.txt


In [14]:
# 3. Feedback helper
import math

# friendly names for a few key joints (you can extend this)
FRIENDLY = {
    "LEFT_ELBOW":     "left elbow",
    "RIGHT_ELBOW":    "right elbow",
    "LEFT_SHOULDER":  "left shoulder",
    "RIGHT_SHOULDER": "right shoulder",
    "LEFT_KNEE":      "left knee",
    "RIGHT_KNEE":     "right knee",
    "SPINE":          "spine",
    "NOSE":           "head",
    # etc…
}

# for each body part keyword, tuple = (verb if err<0, verb if err>0)
ACTION_VERBS = {
    "elbow":    ("flex",        "extend"),
    "shoulder": ("rotate back", "rotate forward"),
    "knee":     ("bend",        "straighten"),
    "hip":      ("lower",       "lift"),
    "spine":    ("arch",        "tuck"),
    # fallback will be ("move", "move")
}

def magnitude_adverb(deg: float) -> str:
    a = abs(deg)
    if a < 5:   return "slightly"
    if a < 15:  return "noticeably"
    return        "significantly"

def get_action(part: str, deg: float) -> str:
    part = part.lower()
    for key, (neg, pos) in ACTION_VERBS.items():
        if key in part:
            return neg if deg < 0 else pos
    return "move"  # generic fallback

def english_feedback(err_vec: list[float], tol: float = 5.0) -> list[str]:
    # pick joints with error > tol
    joints = [(j, deg) for j, deg in enumerate(err_vec) if abs(deg) > tol]
    if not joints:
        return ["✅ Perfect form! Hold that posture."]

    # sort by largest error first, keep top 3
    joints = sorted(joints, key=lambda x: abs(x[1]), reverse=True)[:3]

    tips = []
    for j, deg in joints:
        name = JOINT_NAMES[j]
        part = FRIENDLY.get(name, name.lower().replace("_", " "))
        adv  = magnitude_adverb(deg).capitalize()
        verb = get_action(part, deg)
        tips.append(f"{adv} {verb} your {part} by {abs(deg):.1f}°")
    return tips

# — example usage —
# err_vec = [0, -12.3, 3.1, 20.5, …]  # degrees off for each JOINT_NAMES entry
# print(english_feedback(err_vec))


In [ ]:
# 4. Dataset
class KeypointWindowDataset(Dataset):
    def __init__(self, csv, keypt_root):
        df = pd.read_csv(csv)
        self.rows = df.to_dict('records')
        self.keypt_root = keypt_root

    def __len__(self): return len(self.rows)

    def __getitem__(self,i):
        r = self.rows[i]
        ex, vid = r["exercise_id"], r["video_id"]
        f0, f1 = int(r["window_start"]), int(r["window_end"])
        arr = np.load(next((self.keypt_root/f"Ex{ex}").glob(f"{vid}-Camera17*-mp.npy")))
        seg = arr[f0:f1]                         # (16,33,3)
        seg = seg.reshape(16, -1)                # (16,99)
        seq = torch.from_numpy(seg).float()      # (T,D)

        label = torch.tensor(r["correctness"], dtype=torch.long)
        err   = torch.tensor([r[c] for c in ERR_COLS], dtype=torch.float32)
        return seq, label, err

In [ ]:
# 5. Model definitions
class KeypointEncoder(nn.Module):
    def __init__(self,in_dim,embed=512):
        super().__init__()
        self.conv1 = nn.Conv1d(in_dim,128,3,padding=1)
        self.conv2 = nn.Conv1d(128,embed,3,padding=1)
        self.pool  = nn.AdaptiveAvgPool1d(1)
    def forward(self,x):
        x=x.unsqueeze(2); x=torch.relu(self.conv1(x)); x=torch.relu(self.conv2(x))
        return self.pool(x).squeeze(-1)

class PoseQualityNetKP(nn.Module):
    def __init__(self,in_dim,hidden=256):
        super().__init__()
        self.enc = KeypointEncoder(in_dim)
        self.lstm=nn.LSTM(512,hidden,2,batch_first=True,bidirectional=True)
        dim=hidden*2
        self.cls_head = nn.Linear(dim,2)
        self.err_head = nn.Linear(dim,N_ERR)
    def forward(self,seq):
        feats = torch.stack([self.enc(seq[:,t]) for t in range(seq.size(1))],1)
        out,_ = self.lstm(feats)
        g = out.mean(1)
        return self.cls_head(g), self.err_head(g)


In [ ]:
# pose_quality_windows.py (excerpt)

# … [imports, PoseLandmark, JOINT_TRIPLETS, ERR_JOINTS, Dataset, Model definitions as above] …

def train_epochs(
    csv_file:   str   = "Data-REHAB24-6/Segmentation_windows.csv",
    keypt_root: str   = "Data-REHAB24-6/mp_keypoints",
    epochs:     int   = 30,
    batch:      int   = 16,
    lr:         float = 1e-4,
    ckpt_file:  str   = "kp_pose_quality_windows.pt"
):
    from pathlib import Path
    from torch.utils.data import Subset

    # 1) build dataset and split
    ds = KeypointWindowDataset(Path(csv_file), Path(keypt_root))
    N = len(ds)
    idx = np.arange(N); np.random.shuffle(idx)
    cut1, cut2 = int(0.7*N), int(0.85*N)
    train_idx, val_idx = idx[:cut1], idx[cut1:cut2]

    train_dl = DataLoader(Subset(ds, train_idx), batch_size=batch, shuffle=True)
    val_dl   = DataLoader(Subset(ds, val_idx),   batch_size=batch, shuffle=False)

    # 2) model & losses
    sample_seq, _, _ = ds[0]
    in_dim = sample_seq.shape[-1]
    model  = PoseQualityNetKP(in_dim).to(DEVICE)
    loss_cls = nn.CrossEntropyLoss()
    loss_err = nn.SmoothL1Loss()
    opt      = Adam(model.parameters(), lr)

    best_f1 = 0.0
    for epoch in range(1, epochs+1):
        # — train —
        model.train()
        total_loss = 0.0
        for seq, y, err in tqdm(train_dl, desc=f"Epoch {epoch:02d}"):
            seq, y, err = seq.to(DEVICE), y.to(DEVICE), err.to(DEVICE)
            opt.zero_grad()
            logits, err_hat = model(seq)
            loss = loss_cls(logits, y) + 0.1*loss_err(err_hat, err)
            loss.backward()
            opt.step()
            total_loss += loss.item() * y.size(0)
        print(f"  ↳ train loss: {total_loss/len(train_dl.dataset):.4f}")

        # — val —
        model.eval()
        y_true, y_pred, errs = [], [], []
        with torch.no_grad():
            for seq, y, err in val_dl:
                logits, err_hat = model(seq.to(DEVICE))
                y_true += y.tolist()
                y_pred += logits.argmax(1).cpu().tolist()
                errs    += [ (err_hat.cpu() - err).abs().mean(1) ]
        acc = accuracy_score(y_true, y_pred)
        f1  = f1_score(y_true, y_pred)
        mae = torch.cat(errs).mean().item()
        print(f"  ↳ val acc {acc:.3f}, F1 {f1:.3f}, MAE° {mae:.2f}")

        # — checkpoint —
        if f1 > best_f1:
            best_f1 = f1
            torch.save({
                "state_dict": model.state_dict(),
                "input_dim":  in_dim,
                "hidden":     model.lstm.hidden_size,
                "best_f1":    best_f1
            }, ckpt_file)
            print(f"  ✓ saved new best model to {ckpt_file} (F1 {f1:.3f})")

In [ ]:
# 8. Run training
train_epochs(
        csv_file   = "Data-REHAB24-6/Segmentation_windows.csv",
        keypt_root = "Data-REHAB24-6/mp_keypoints",
        epochs     = 50,
        batch      = 16,
        lr         = 1e-4,
        ckpt_file  = "kp_pose_quality_windows.pt"
    )

/var/folders/pv/99z70cjs7t1dx16_0r1c33hw0000gn/T/ipykernel_23368/1680097000.py:19: UserWarning: err_0…err_NJOINTS not found → filling zeros
  warnings.warn("err_0…err_NJOINTS not found → filling zeros")
/var/folders/pv/99z70cjs7t1dx16_0r1c33hw0000gn/T/ipykernel_23368/1680097000.py:19: UserWarning: err_0…err_NJOINTS not found → filling zeros
  warnings.warn("err_0…err_NJOINTS not found → filling zeros")
Epoch 01: 100%|██████████| 46/46 [00:01<00:00, 36.69it/s]


  ↳ train loss: 0.6925
  ↳ val acc 0.486, F1 0.654, MAE° 0.00
  ✓ saved new best model to kp_pose_quality_mp.pt  (epoch 1, F1 0.654)


Epoch 02: 100%|██████████| 46/46 [00:00<00:00, 50.56it/s]


  ↳ train loss: 0.6935
  ↳ val acc 0.486, F1 0.654, MAE° 0.00


Epoch 03: 100%|██████████| 46/46 [00:00<00:00, 46.31it/s]


...

  ↳ train loss: 0.6623
  ↳ val acc 0.543, F1 0.652, MAE° 0.00


Epoch 49: 100%|██████████| 46/46 [00:00<00:00, 49.93it/s]


  ↳ train loss: 0.6586
  ↳ val acc 0.549, F1 0.530, MAE° 0.00


Epoch 50: 100%|██████████| 46/46 [00:00<00:00, 49.96it/s]


  ↳ train loss: 0.6618
  ↳ val acc 0.543, F1 0.578, MAE° 0.00


In [19]:
import cv2
import numpy as np
import torch
from collections import deque
import mediapipe as mp

# MediaPipe setup
mp_drawing = mp.solutions.drawing_utils
mp_styles  = mp.solutions.drawing_styles
mp_pose    = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Load your feedback helper
# from your_feedback_cell import english_feedback

# Rebuild model (use weights_only=True to suppress the pickle warning)
ckpt = torch.load('kp_pose_quality_mp.pt', map_location=DEVICE, weights_only=True)
in_dim, hidden, frames = ckpt["input_dim"], ckpt["hidden"], ckpt["frames"]
infer_model = PoseQualityNetKP(in_dim, hidden).to(DEVICE)
infer_model.load_state_dict(ckpt["state_dict"])
infer_model.eval()
print("✅ Model loaded. Starting live inference…")

cap = cv2.VideoCapture(0)
buffer = deque(maxlen=frames)
frame_count = 0

print("▶ Press ESC to exit.")
while True:
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        print("⚠️  Failed to grab frame, exiting.")
        break

    # Prepare image for MediaPipe
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False
    results = mp_pose.process(rgb)  # IMAGE_DIMENSIONS warning can be ignored
    rgb.flags.writeable = True

    img = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        # Draw skeleton
        mp_drawing.draw_landmarks(
            img, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style()
        )
        # Extract and buffer keypoints
        arr = np.array([[lm.x, lm.y, lm.z]
                        for lm in results.pose_landmarks.landmark],
                       dtype=np.float32).reshape(-1)
        buffer.append(arr)
        print(f"[Frame {frame_count}] Buffered {len(buffer)}/{frames}")
    else:
        print(f"[Frame {frame_count}] No landmarks")

    # When buffer is full, run inference
    if len(buffer) == frames:
        print("▶ Buffer full — running inference")
        seq = torch.tensor(np.stack(buffer), dtype=torch.float32).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            logits, err_hat = infer_model(seq)
        # Debug print raw outputs
        print("   raw logits:", logits.cpu().numpy())
        print("   raw err (first 5):", err_hat.squeeze(0).cpu().numpy()[:5])

        pred = logits.argmax(1).item()
        tips = english_feedback(err_hat.squeeze(0).cpu().numpy())
        print(f"   🎯 pred={pred}, tips={tips}")

        # Overlay tips on image
        color = (0,255,0) if pred == 1 else (0,0,255)
        for i, tip in enumerate(tips):
            cv2.putText(img, tip, (10, 30 + 30*i),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        # Clear buffer so next window is fresh
        buffer.clear()

    else:
        # Show buffering progress
        cv2.putText(img,
                    f"Buffering keypoints: {len(buffer)}/{frames}",
                    (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (200,200,0), 2, cv2.LINE_AA)

    cv2.imshow("Live Pose Feedback", img)
    if cv2.waitKey(1) & 0xFF == 27:
        print("❎ ESC pressed, exiting.")
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1744977798.436367  716137 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Max
W0000 00:00:1744977798.487191  734105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744977798.509922  734111 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ Model loaded. Starting live inference…
▶ Press ESC to exit.
[Frame 1] No landmarks
[Frame 2] Buffered 1/16
[Frame 3] Buffered 2/16
[Frame 4] Buffered 3/16
[Frame 5] Buffered 4/16
[Frame 6] Buffered 5/16
[Frame 7] Buffered 6/16
[Frame 8] Buffered 7/16
[Frame 9] Buffered 8/16
[Frame 10] Buffered 9/16
[Frame 11] Buffered 10/16
[Frame 12] Buffered 11/16
[Frame 13] Buffered 12/16
[Frame 14] Buffered 13/16
[Frame 15] Buffered 14/16
[Frame 16] Buffered 15/16
[Frame 17] Buffered 16/16
▶ Buffer full — running inference
   raw logits: [[-1.0874856  1.0430459]]
   raw err (first 5): [-0.00043773  0.00259034 -0.00691423  0.01132228  0.00314515]
   🎯 pred=1, tips=['✅ Perfect form! Hold that posture.']
[Frame 18] Buffered 1/16
[Frame 19] Buffered 2/16
[Frame 20] Buffered 3/16
[Frame 21] Buffered 4/16
[Frame 22] Buffered 5/16
[Frame 23] Buffered 6/16
[Frame 24] Buffered 7/16
❎ ESC pressed, exiting.
